In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import json
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

2025-10-10 22:41:42.293228: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-10 22:41:42.831558: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-10 22:41:45.054196: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
MAX_PROT_LEN = 1200
MAX_DRUG_LEN = 85
EMBED_DIM = 128

In [3]:
DATA_PATH = r"/teamspace/studios/this_studio/datasets/kiba.txt"

df = pd.read_csv(
    DATA_PATH, 
    sep=r'\s+',  # one or more spaces/tabs
    header=None, 
    engine="python",
    names=["drug_id", "protein_id", "smiles", "sequence", "affinity"]
)

print(f"Loaded {len(df)} samples from {DATA_PATH}")
print(df.head())


Loaded 118254 samples from /teamspace/studios/this_studio/datasets/kiba.txt
         drug_id protein_id                                         smiles  \
0  CHEMBL1087421     O00141  COC1=C(C=C2C(=C1)CCN=C2C3=CC(=C(C=C3)Cl)Cl)Cl   
1  CHEMBL1087421     O14920  COC1=C(C=C2C(=C1)CCN=C2C3=CC(=C(C=C3)Cl)Cl)Cl   
2  CHEMBL1087421     O15111  COC1=C(C=C2C(=C1)CCN=C2C3=CC(=C(C=C3)Cl)Cl)Cl   
3  CHEMBL1087421     P00533  COC1=C(C=C2C(=C1)CCN=C2C3=CC(=C(C=C3)Cl)Cl)Cl   
4  CHEMBL1087421     P04626  COC1=C(C=C2C(=C1)CCN=C2C3=CC(=C(C=C3)Cl)Cl)Cl   

                                            sequence  affinity  
0  MTVKTEAAKGTLTYSRMRGMVAILIAFMKQRRMGLNDFIQKIANNS...      11.1  
1  MSWSPSLTTQTCGAWEMKERLGTGGFGNVIRWHNQETGEQIAIKQC...      11.1  
2  MERPPGLRPGAGGPWEMRERLGTGGFGNVCLYQHRELDLKIAIKSC...      11.1  
3  MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...      11.1  
4  MELAALCRWGLLLALLPPGAASTQVCTGTDMKLRLPASPETHLDML...      11.1  


In [4]:
all_smiles_chars = set("".join(df["smiles"].astype(str).values))
all_smiles_chars = sorted(all_smiles_chars)

smiles_token2idx = {}
for index, char in enumerate(all_smiles_chars):
    smiles_token2idx[char] = index + 1

all_protein_aas = set("".join(df["sequence"].astype(str).values))
all_protein_aas = sorted(all_protein_aas)

protein_token2idx = {}
for index, aa in enumerate(all_protein_aas):
    protein_token2idx[aa] = index + 1

def tokenize_smiles(smiles_string, max_len=MAX_DRUG_LEN):
    tokens = []
    for ch in smiles_string[:max_len]:
        if ch in smiles_token2idx:
            tokens.append(smiles_token2idx[ch])

    padded_tokens = np.pad(tokens, pad_width=(0, max_len - len(tokens)), mode="constant",constant_values=0)

    return padded_tokens

def tokenize_protein(sequence_string, max_len=MAX_PROT_LEN):
    tokens = []
    for ch in sequence_string[:max_len]:
        if ch in protein_token2idx:
            tokens.append(protein_token2idx[ch])

    padded_tokens = np.pad(tokens, pad_width=(0, max_len - len(tokens)), mode="constant",constant_values=0)

    return padded_tokens

drug_token_arrays = []
for smiles in df["smiles"]:
    token_array = tokenize_smiles(smiles)
    drug_token_arrays.append(token_array)

protein_token_arrays = []
for seq in df["sequence"]:
    token_array = tokenize_protein(seq)
    protein_token_arrays.append(token_array)

X_drug = np.vstack(drug_token_arrays)
X_prot = np.vstack(protein_token_arrays)

y = df["affinity"].values.astype(np.float32).reshape(-1, 1)

print("Drugs:", X_drug.shape)
print("Proteins:", X_prot.shape)
print("Affinities:", y.shape)

Drugs: (118254, 85)
Proteins: (118254, 1200)
Affinities: (118254, 1)


In [5]:
X_prot_train, X_prot_test, X_drug_train, X_drug_test, y_train, y_test = train_test_split(
    X_prot, X_drug, y, test_size=0.2, random_state=42, shuffle=True
)
print("Train size:", len(y_train))
print("Test size:", len(y_test))

Train size: 94603
Test size: 23651


In [6]:
protein_input = Input(shape=(MAX_PROT_LEN,), name="protein_input")
p_embed = Embedding(input_dim=len(all_protein_aas) + 1, output_dim=EMBED_DIM)(protein_input)
p_conv1 = Conv1D(32, 4, activation="relu", padding="valid")(p_embed)
p_conv2 = Conv1D(64, 6, activation="relu", padding="valid")(p_conv1)
p_conv3 = Conv1D(96, 8, activation="relu", padding="valid")(p_conv2)
p_flat = GlobalMaxPooling1D()(p_conv3)

drug_input = Input(shape=(MAX_DRUG_LEN,), name="drug_input")
d_embed = Embedding(input_dim=len(all_smiles_chars) + 1, output_dim=EMBED_DIM)(drug_input)
d_conv1 = Conv1D(32, 4, activation="relu", padding="valid")(d_embed)
d_conv2 = Conv1D(64, 6, activation="relu", padding="valid")(d_conv1)
d_conv3 = Conv1D(96, 8, activation="relu", padding="valid")(d_conv2)
d_flat = GlobalMaxPooling1D()(d_conv3)

merged = Concatenate()([p_flat, d_flat])
dense1 = Dense(1024, activation="relu")(merged)
drop1 = Dropout(0.1)(dense1)
dense2 = Dense(1024, activation="relu")(drop1)
drop2 = Dropout(0.1)(dense2)
dense3 = Dense(512, activation="relu")(drop2)
output = Dense(1, activation="linear")(dense3)

model = Model(inputs=[protein_input, drug_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-4), loss="mse")

model.summary()

I0000 00:00:1760136128.103272    2671 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21087 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:04.0, compute capability: 8.9


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ protein_input       │ (None, 1200)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ drug_input          │ (None, 85)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1200, 128) │      2,688 │ protein_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 85, 128)   │      3,840 │ drug_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 1197, 32)  │     16,416 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 82, 32)    │     16,416 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 1192, 64)  │     12,352 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 77, 64)    │     12,352 │ conv1d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 1185, 96)  │     49,248 │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 70, 96)    │     49,248 │ conv1d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 96)        │          0 │ conv1d_2[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 96)        │          0 │ conv1d_5[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 192)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1024)      │    197,632 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1024)      │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1024)      │  1,049,600 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 1024)      │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 512)       │    524,800 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │        513 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,935,105 (7.38 MB)

 Trainable params: 1,935,105 (7.38 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
early_stop = EarlyStopping(monitor="val_loss", patience=30, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, min_lr=1e-6)

history = model.fit(
    [X_prot_train, X_drug_train],
    y_train,
    validation_split=0.1,
    epochs=100,
    batch_size=64,
    callbacks=[early_stop, reduce_lr],
    verbose=1,
)

Epoch 1/100


2025-10-10 22:42:14.161734: I external/local_xla/xla/service/service.cc:163] XLA service 0x758ca4005cf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-10 22:42:14.161759: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9
2025-10-10 22:42:14.266904: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-10 22:42:14.864260: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
2025-10-10 22:42:15.146919: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-10-10 22:42:17.116645: I external/local_

  15/1331 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - loss: 134.9030 

I0000 00:00:1760136144.764486    8179 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1330/1331 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 12.5812

2025-10-10 22:42:41.288653: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1832', 4 bytes spill stores, 4 bytes spill loads



1331/1331 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 12.5741

2025-10-10 22:42:48.572749: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_145', 8 bytes spill stores, 8 bytes spill loads

2025-10-10 22:42:49.512218: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_145', 1064 bytes spill stores, 1068 bytes spill loads

2025-10-10 22:42:50.778561: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-10-10 22:42:51.858757: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_145', 264 by

1331/1331 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - loss: 3.0643 - val_loss: 0.5461 - learning_rate: 1.0000e-04
Epoch 2/100
1331/1331 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.5750 - val_loss: 0.4858 - learning_rate: 1.0000e-04
Epoch 3/100
1331/1331 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.5302 - val_loss: 0.4612 - learning_rate: 1.0000e-04
Epoch 4/100
1331/1331 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.5038 - val_loss: 0.4545 - learning_rate: 1.0000e-04
Epoch 5/100
1331/1331 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.4762 - val_loss: 0.4592 - learning_rate: 1.0000e-04
Epoch 6/100
1331/1331 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.4531 - val_loss: 0.4587 - learning_rate: 1.0000e-04
Epoch 7/100
1331/1331 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.4283 - val_loss: 0.4449 - learning_rate: 1.0000e-04
Epoch 8/100
1331/1331 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.4165 - val_loss: 0.4037 - learning_rate: 1.0000e-04
Epoch 9/100
1331/1331 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - l

In [8]:
y_pred = model.predict([X_prot_test, X_drug_test])
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)

def concordance_index(y_true, y_pred):
    pairs = conc = 0
    for i in range(len(y_true)):
        for j in range(i + 1, len(y_true)):
            if y_true[i] != y_true[j]:
                pairs += 1
                if (y_pred[i] > y_pred[j] and y_true[i] > y_true[j]) or (y_pred[i] < y_pred[j] and y_true[i] < y_true[j]):
                    conc += 1
                elif y_pred[i] == y_pred[j]:
                    conc += 0.5
    return conc / pairs if pairs > 0 else 0

ci = concordance_index(y_test.ravel(), y_pred.ravel())

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"CI: {ci:.4f}")

740/740 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step
MSE: 0.1689
RMSE: 0.4110
CI: 0.8675


In [10]:
SAVE_DIR = r"/teamspace/studios/this_studio/models/kiba_better"
os.makedirs(SAVE_DIR, exist_ok=True)

model_save_path = os.path.join(SAVE_DIR, "deepdta_kiba_model.keras")
model.save(model_save_path)
print(f"Model saved at: {model_save_path}")

weights_path = os.path.join(SAVE_DIR, "deepdta_kiba.weights.h5")
model.save_weights(weights_path)
print(f"Weights saved at: {weights_path}")

tokenizers_path = os.path.join(SAVE_DIR, "tokenizers.pkl")
with open(tokenizers_path, "wb") as f:
    pickle.dump({
        "smiles_token2idx": smiles_token2idx,
        "protein_token2idx": protein_token2idx
    }, f)
print(f"Tokenizers saved at: {tokenizers_path}")

history_path = os.path.join(SAVE_DIR, "training_history.json")
with open(history_path, "w") as f:
    json.dump(history.history, f)
print(f"Training history saved at: {history_path}")

results_path = os.path.join(SAVE_DIR, "results.json")
results = {
    "mse": float(mse),
    "rmse": float(rmse),
    "ci": float(ci)
}
with open(results_path, "w") as f:
    json.dump(results, f, indent=4)
print(f"Results saved at: {results_path}")

preds_path = os.path.join(SAVE_DIR, "predictions.csv")
pd.DataFrame({
    "y_true": y_test.ravel(),
    "y_pred": y_pred.ravel()
}).to_csv(preds_path, index=False)
print(f"Predictions saved at: {preds_path}")


Model saved at: /teamspace/studios/this_studio/models/kiba_better/deepdta_kiba_model.keras
Weights saved at: /teamspace/studios/this_studio/models/kiba_better/deepdta_kiba.weights.h5
Tokenizers saved at: /teamspace/studios/this_studio/models/kiba_better/tokenizers.pkl
Training history saved at: /teamspace/studios/this_studio/models/kiba_better/training_history.json
Results saved at: /teamspace/studios/this_studio/models/kiba_better/results.json
Predictions saved at: /teamspace/studios/this_studio/models/kiba_better/predictions.csv
